Я вижу два решения:

Для простоты я бы сделал 2 класса датасета
Если нужно кодировать лишь последовательность букв, он и хранит последовательности букв сразу и не хранит коордианты

В обоих случаях декодер оперирует эмбеддингами букв текста

<h3>1. На вход энкодера x, y, t, dx/dt, dy/st, x'', y'', keybard_key_embedding</h3>

**Что делать, если ближайшая клавиша неалфавитная (пунктуация, клавиши-действия)?**

Добавлю для всех неалфавитных клавиш один специальный токен. Хотя, возможно,
лучше добавить отдельный токен для каждой клавиши. Кажется, в кавиатуре схожесть
клавиш определяется тем, насколько часто они встречаются рядом друг с другом.
Тогда может быть важно отличать enter, который близок к бкве `э`, например,
от `caps lock`, который близок к `ф`. Отмечу, что схожесть в данном случае - это
не просто физическое расстояние между клавишами (хотя отчасти и так), но скорее, похожесть
клавиш `a` и `b` означает, что для последовательностей вида `letter1`, `letter2` ... `letterX`, `letterN`
если вероятность, что `letterX` = `a` велика, то вероятность, что `letterX` = `b` тоже велика.


**Где происходит инициализация токенизатора?**
я бы вынес токенезатор вне датасета и передавал бы его в конструктор датасета.


для каждой раскладки свои instance'ы датасета и модели.



<h3> 2. На вход энкодера последовательность клавиш клавиатуры </h3>

Если ближайшая клавиша неалфавитная **пропускать**

**Где происходит инициализация токенизатора?**


один instance датасета и одна модель для всех раскладок.


Реализовывать ли для каждого варианта отдельный токенизатор:

У нас может быть различное количество токенов: в некотоорых раскладках отсутствует символ "ъ", например

Когда датасет содержит лишь одну раскладку, токенизатор должен учесть символы из одной раскладки. Когда датасет содержит несколько раскладок, токенизатор должен учесть символы из всех раскладок.

Кажется, что варьируется только наличие 'ъ' и 'ё'. Во-первых, не ясно нужны ли эти символы. Есть желание заменять 'ё' на 'е', а 'ъ' на 'ь'. 

Кажется, нас совершенно устраивает токенизатор, содержащий все буквы русского языка, включая 'ё' и 'ъ'. Наличие пары лишних токенов незначительно увеличит размер эмбеддинг-матрицы, но не повлияет на обучение модели.

In [362]:
%load_ext autoreload
%autoreload 2

In [300]:
import os

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, IterableDataset

# from model import SwipeCurveEncoderTransformer

In [306]:
IN_KAGGLE = False

In [307]:
if IN_KAGGLE:
    DATA_ROOT = "/kaggle/input/yandex-cup-playground"
    MODELS_DIR = ""
else:
    DATA_ROOT = "../data/data_separated_grid"
    MODELS_DIR = "../data/trained_models/m1"

In [308]:
from typing import List, Optional

class CharLevelTokenizerv1:
    """
    Tokenizes a word into a list of integers.

    Toknized word is padded to the max_word_len.

    Guarantees that <sos> and <pad> are tokens with `vocab_len - 1`
    and `vocab_len - 2` indices respectively.
    This is useful because model never generates <sos> and <pad> tokens.
    """
    def __init__(self, vocab_path):
        self.char_to_idx = {}
        self.idx_to_char = {}
        self.max_word_len = None  # is set in _build_vocab
        self._build_vocab(vocab_path)

    def _build_vocab(self, vocab_path):
        self.max_word_len = 0
        special_tokens = ["<eos>", "<pad>", "<unk>", "<sos>"]
        unique_chars = set()

        with open(vocab_path, "r", encoding="utf-8") as f:
            vocab = f.read().split("\n")
            for word in vocab:
                self.max_word_len = max(self.max_word_len, len(word) + 2)  # + <sos> and <eos>
                for char in word:
                    unique_chars.add(char)
                    
        unique_chars_list = sorted(list(unique_chars)) + special_tokens
        self.char_to_idx = {char: idx for idx, char in enumerate(unique_chars_list)}
        self.idx_to_char = {idx: char for idx, char in enumerate(unique_chars_list)}

    def _tokenize_word(self, word):
        """
        Tokenizes a word into a list of integers.
        """
        tokenized_word = []
        tokenized_word.append(self.char_to_idx["<sos>"])
        for char in word:
            default: int = self.char_to_idx['<unk>']
            tokenized_word.append(self.char_to_idx.get(char, default))
        tokenized_word.append(self.char_to_idx["<eos>"])
        return tokenized_word
    
    
    def _pad(self, tokenized_word):
        """
        Pads a word to the max_word_len.
        """
        return tokenized_word + [self.char_to_idx["<pad>"]] * (self.max_word_len - len(tokenized_word))
    
    def tokenize(self, word):
        """
        Tokenizes a word and pads it to the max_word_len.
        """
        token_seq = torch.tensor(self._pad(self._tokenize_word(word)))
        mask = torch.ones(self.max_word_len, dtype=torch.bool)
        mask[:len(word)+2] = False
        return token_seq, mask
    
    def decode(self, token_seq):
        """
        Decodes a tokenized word into a string.
        """
        return "".join([self.idx_to_char[int(idx)] for idx in token_seq])

In [309]:
import pickle

word_tokenizer = CharLevelTokenizerv1(os.path.join(DATA_ROOT, "voc.txt"))

if not IN_KAGGLE:
    word_tokenizer_save_path = os.path.join(DATA_ROOT, "word_tokenizer.pkl")

    with open(word_tokenizer_save_path, 'wb') as f:
        pickle.dump(word_tokenizer, f, protocol=pickle.HIGHEST_PROTOCOL)

    with open(word_tokenizer_save_path, 'rb') as f:
        word_tokenizer = pickle.load(f)

In [310]:
print(word_tokenizer.idx_to_char)

{0: '-', 1: 'а', 2: 'б', 3: 'в', 4: 'г', 5: 'д', 6: 'е', 7: 'ж', 8: 'з', 9: 'и', 10: 'й', 11: 'к', 12: 'л', 13: 'м', 14: 'н', 15: 'о', 16: 'п', 17: 'р', 18: 'с', 19: 'т', 20: 'у', 21: 'ф', 22: 'х', 23: 'ц', 24: 'ч', 25: 'ш', 26: 'щ', 27: 'ъ', 28: 'ы', 29: 'ь', 30: 'э', 31: 'ю', 32: 'я', 33: '<eos>', 34: '<pad>', 35: '<unk>', 36: '<sos>'}


In [311]:
class KeyboardTokenizerv1:
    
    i2t = ['а', 'б', 'в', 'г', 'д', 'е', 'ë', 'ж', 'з', 'и', 'й',
           'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф',
           'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я',
           '-', '<unk>', '<pad>']
    
    t2i = {t: i for i, t in enumerate(i2t)}

    def get_token(self, char):
        return self.t2i.get(char, self.t2i['<unk>'])

In [312]:
import json
from typing import Optional, List, Tuple, Dict
import array
import numpy as np

import torch
from torch.utils.data import Dataset
from tqdm import tqdm


class NeuroSwipeDatasetv1(Dataset):
    """
    Dataset class for NeuroSwipe dataset.
    The dataset file weights over 3 GB and contains over 6 million swipe gestures.
    """

    def __init__(self,
                 data_path: str,
                 gridname_to_grid: dict,
                 kb_tokenizer: KeyboardTokenizerv1,
                 max_traj_len: int,
                 word_tokenizer: CharLevelTokenizerv1,  # should contain max word len
                 include_time: bool = False,
                 include_velocities: bool = True,
                 include_accelerations: bool = True,
                 include_grid_name: bool = False,
                 has_target = True,
                 has_one_grid_only = True,
                 total: Optional[int] = None):
        """
        Argsuments:
        -----------
        data_path (string): Path to the NeuroSwipe dataset in JSON format.
            A custom version of the dataset is used:
            "grid" property is replaced with "grid_name" property.
        """
        if include_accelerations and not include_velocities:
            raise ValueError("Accelerations are supposed \
                             to be an addition to velocities. Add velocities.")
        
        if has_one_grid_only and len(gridname_to_grid) != 1:
            raise ValueError(f"has_one_grid_only is True \
                             but len(gridname_to_grid) != 1")

        self.max_traj_len = max_traj_len
        self.include_velocities = include_velocities
        self.include_accelerations = include_accelerations
        self.include_time = include_time
        self.has_target = has_target
        self.include_grid_name = include_grid_name

        self.word_tokenizer = word_tokenizer

        self._grid_name_to_grid = gridname_to_grid

        self._nearest_kb_label_dict = self._get_nearest_kb_label_dict(gridname_to_grid)

        self.data_list = []
        self._set_data(data_path, gridname_to_grid, kb_tokenizer, self.data_list, total = total)


    def get_nearest_kb_label(self, x, y, grid_name, gridname_to_grid):
        """
        Given coords on a keyboard (x, y) and its grid_name returns the nearest keyboard keys

        By default it uses an array assosiated with grid_name
        that stores the nearest key label for every possible coord pair.

        However sometimes the coords are outside of the keyboard boarders.
        In this case we find the nearest key in a loop.
        """        
        grid = gridname_to_grid[grid_name]
        if x < 0 or x >= grid['width'] or y < 0 or y >= grid['height']:
            return self._get_kb_label_without_map(x, y, grid)
        else:
            return self._nearest_kb_label_dict[grid_name][x, y]
    


    def _get_key_center(self, hitbox: Dict[str, int]) -> Tuple[int, int]:
        x = hitbox['x'] + hitbox['w'] / 2
        y = hitbox['y'] + hitbox['h'] / 2
        return x, y
    

    def _get_kb_label_without_map(self, x, y, grid: dict) -> str:
        """
        Returns label of the nearest key on the keyboard without using a map.
         
        Iterates over all keys and calculates the distance to (x, y) to find the nearest one.
        """
        nearest_kb_label = None
        min_dist = float("inf")
        for key in grid['keys']:
            key_x, key_y = self._get_key_center(key['hitbox'])
            dist = (x - key_x)**2 + (y - key_y)**2
            if dist < min_dist:
                min_dist = dist
                if 'label' in key:
                    nearest_kb_label = key['label']
                elif 'action' in key:
                    nearest_kb_label = key['action']  # tokenizer will covert it to <unk>
                else:
                    raise ValueError("Key has no label or action")
        return nearest_kb_label


    def _get_nearest_kb_label_dict(self, gridname_to_grid: dict) -> Dict[str, np.chararray]:
        """
        Creates a dict that maps grid_name to a map (np.chararray) from coordinate to nearest key label.
        """
        nearest_kb_label_dict = {}
        for grid_name, grid in gridname_to_grid.items():
            nearest_kb_label_dict[grid_name] = self._get_coord_to_kb_label(grid)
        return nearest_kb_label_dict
    

    def _get_coord_to_kb_label(self, grid: dict) -> np.chararray:
        coord_to_kb_label = np.chararray(
            (grid['width'], grid['height']), unicode=True) # 1080 x 640 in our case
        coord_to_kb_label.fill('')

        for key in grid['keys']:
            x_left = key['hitbox']['x']
            x_right = x_left + key['hitbox']['w']
            y_top = key['hitbox']['y']
            y_bottom = y_top + key['hitbox']['h']

            # tokenizer will covert actions to <unk>
            label = key['label'] if 'label' in key else key['action']

            if len(label) > 1:
                print(f"Warning: label '{label}' is substituted with {label[0]}")

            coord_to_kb_label[x_left:x_right, y_top:y_bottom] = label


        for x in range(grid['width']):
            for y in range(grid['height']):
                if coord_to_kb_label[x, y] != '':
                    continue
                coord_to_kb_label[x, y] = self._get_kb_label_without_map(x, y, grid)

        return coord_to_kb_label
            

    def _set_data(self,
                  data_path: str,
                  gridname_to_grid: dict,
                  kb_tokenizer,
                  data_list: list,
                  total: Optional[int] = None):
        with open(data_path, "r", encoding="utf-8") as json_file:
            for line in tqdm(json_file, total = total):
                data_list.append(self._get_data_from_json_line(line, gridname_to_grid, kb_tokenizer))


    def _get_dx_dt(self,
                   X: torch.tensor,
                   T: torch.tensor,
                   len: int) -> List[float]:
        """
        Calculates dx/dt for a list of x coordinates and a list of t coordinates.

        Arguments:
        ----------
        X : torch.tensor
            x (position) coordinates.
        T : torch.tensor
            T[i] = time from the beginning of the swipe corresponding to X[i].
        len : int
            Length of the swipe trajectory. Indexes greater than len are ignored.
        """
        dx_dt = torch.zeros_like(X)
        # dx_dt[1:-1] = (X[2:] - X[:-2]) / (T[2:] - T[:-2])
        dx_dt[1:len-1] = (X[2:len] - X[:len-2]) / (T[2:len] - T[:len-2])

        # Example:
        # x0 x1 x2 x3
        # t0 t1 t2 t3
        # dx_dt[0] = 0
        # dx_dt[1] = (x2 - x0) / (t2 - t0)
        # dx_dt[2] = (x3 - x1) / (t3 - t1)
        # dx_dt[3] = 0


        # if True in torch.isnan(dx_dt):
        #     print(dx_dt)
        #     raise ValueError("dx_dt contains NaNs")

        return dx_dt
    
    def _get_data_from_json_line(self,
                                 line,
                                 gridname_to_grid,
                                 kb_tokenizer) -> Tuple[list, list, list, str]:
        """
        Parses a JSON line and returns a dictionary with data.
        """
        data = json.loads(line)

        X = array.array('h', data['curve']['x'])
        Y = array.array('h', data['curve']['y'])
        T = array.array('h', data['curve']['t'])

        grid_name = data['curve']['grid_name']   

        kb_labels = [self.get_nearest_kb_label(x, y, grid_name, gridname_to_grid) for x, y in zip(X, Y)]
        kb_tokens = [kb_tokenizer.get_token(label) for label in kb_labels]
        kb_tokens += [kb_tokenizer.get_token('<pad>')] * (self.max_traj_len - len(kb_labels))
        kb_tokens = array.array('h', kb_tokens)

        if not self.has_target:
            return X, Y, T, kb_tokens, grid_name
        else:
            word: str = data['word']
            return X, Y, T, kb_tokens, word, grid_name


    def __len__(self):
        return len(self.data_list)
    
    def __getitem__(self, idx):
        if self.has_target:
            X_list, Y_list, T_list, kb_tokens, word, grid_name = self.data_list[idx]
        else:
            X_list, Y_list, T_list, kb_tokens, grid_name = self.data_list[idx]

        X = torch.zeros(self.max_traj_len, dtype=torch.float32)
        Y = torch.zeros(self.max_traj_len, dtype=torch.float32)
        T = torch.zeros(self.max_traj_len, dtype=torch.float32)

        grid = self._grid_name_to_grid[grid_name]

        X[:len(X_list)] = torch.tensor(X_list, dtype=torch.float32) / grid['width']
        Y[:len(Y_list)] = torch.tensor(Y_list, dtype=torch.float32) / grid['height']
        T[:len(T_list)] = torch.tensor(T_list, dtype=torch.float32)

        xyt = torch.cat(
            (
                X.reshape(-1, 1),
                Y.reshape(-1, 1),
            ),
            axis = 1
        )

        if self.include_time:
            xyt = torch.cat(
                (
                    xyt,
                    T.reshape(-1, 1)
                ),
                axis = 1
            )

        traj_len = len(X_list)

        if self.include_velocities:
            dx_dt = self._get_dx_dt(X, T, traj_len)
            dy_dt = self._get_dx_dt(Y, T, traj_len)
            xyt = torch.cat(
                [
                    xyt,
                    dx_dt.reshape(-1, 1),
                    dy_dt.reshape(-1, 1)
                ],
                axis = 1
            )

        if self.include_accelerations:
            d2x_dt2 = self._get_dx_dt(dx_dt, T, traj_len)
            d2y_dt2 = self._get_dx_dt(dy_dt, T, traj_len)
            xyt = torch.cat(
                [
                    xyt,
                    d2x_dt2.reshape(-1, 1),
                    d2y_dt2.reshape(-1, 1)
                ],
                axis = 1
            )

        traj_pad_mask = torch.ones(self.max_traj_len, dtype=torch.bool)
        traj_pad_mask[:len(X_list)] = False


        kb_tokens = torch.tensor(kb_tokens, dtype=torch.int64)

        if self.has_target:
            char_seq, word_mask = self.word_tokenizer.tokenize(word)
            word_mask = word_mask[:-1]

            decoder_in_char_seq = char_seq[:-1]
            decoder_out_char_seq = char_seq[1:]

        
        if not self.has_target:
            decoder_out_char_seq = None
            decoder_in_char_seq = None
            word_mask = None
        
        if self.include_grid_name:
            return (xyt, kb_tokens, decoder_in_char_seq, traj_pad_mask, word_mask), decoder_out_char_seq, grid_name
        
        return (xyt, kb_tokens, decoder_in_char_seq, traj_pad_mask, word_mask), decoder_out_char_seq

In [313]:
def get_grid(grid_name: str, grids_path: str) -> dict:
    with open(grids_path, "r", encoding="utf-8") as f:
        return json.load(f)[grid_name]

In [314]:
sample_data = os.path.join(DATA_ROOT, "valid__in_train_format__default_only.jsonl")
grid_path =  os.path.join(DATA_ROOT, "gridname_to_grid.json")
grid_name = "default"

grid = get_grid(grid_name, grid_path)
kb_tokenizer = KeyboardTokenizerv1()
word_tokenizer = CharLevelTokenizerv1(os.path.join(DATA_ROOT, "voc.txt"))


dataset = NeuroSwipeDatasetv1(
    data_path = sample_data,
    gridname_to_grid= {grid_name: grid},
    kb_tokenizer = kb_tokenizer,
    max_traj_len = 299,
    word_tokenizer = word_tokenizer,
    include_time = False,
    include_velocities = True,
    include_accelerations = True,
    total = 9_416
)

100%|██████████| 9416/9416 [00:03<00:00, 2817.57it/s]


In [12]:
# full train dataset before adding nearest_kb_label:
# ----------------------------------
# 16.2GB RAM (when data stored as torch tensors)
# when stored as lists 17Gb is 61%
# when data stored as python arrays with dtype short: 4.8GB RAM


# Now:
# ----
# 9.3 Gb RAM

In [315]:
i = 40
(xyt, kb_tokens, dec_in_char_seq, traj_pad_mask, word_pad_mask), dec_out_char_seq = dataset[i]
print(xyt.shape, kb_tokens.shape, dec_in_char_seq.shape, traj_pad_mask.shape, word_pad_mask.shape, dec_out_char_seq.shape)

torch.Size([299, 6]) torch.Size([299]) torch.Size([35]) torch.Size([299]) torch.Size([35]) torch.Size([35])


In [344]:
with np.printoptions(threshold=1000000):
    print(dataset._nearest_kb_label_dict['default'][:5])

[['й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й'
  'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й'
  'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й'
  'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й'
  'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й'
  'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й'
  'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й'
  'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й'
  'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й'
  'й' 'й' 'й' 'й' 'й' 'й' 'й' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф'
  'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф'
  'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф'
  'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф'
  'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 

In [318]:
# def get_mask(max_seq_len: int):
#     """
#     Returns a mask for the decoder transformer.
#     """
#     mask = torch.triu(torch.ones(max_seq_len, max_seq_len), diagonal=1)
#     mask = mask.masked_fill(mask == 1, float('-inf'))
#     return mask

# mask = get_mask(5)
# print(mask)

# >>>
# tensor([[0., -inf, -inf, -inf, -inf],
#         [0., 0., -inf, -inf, -inf],
#         [0., 0., 0., -inf, -inf],
#         [0., 0., 0., 0., -inf],
#         [0., 0., 0., 0., 0.]])

In [319]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [371]:
from model import SwipeCurveTransformer


In [369]:
!python model_tests.py

Encoder out shape correct
Decoder out shape correct


c:\Users\proshian\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


In [323]:
word_char_tokenizer = CharLevelTokenizerv1(os.path.join(DATA_ROOT, "voc.txt"))

In [324]:
word_char_tokenizer.tokenize("троллейбус")

(tensor([36, 19, 17, 15, 12, 12,  6, 10,  2, 20, 18, 33, 34, 34, 34, 34, 34, 34,
         34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34]),
 tensor([False, False, False, False, False, False, False, False, False, False,
         False, False,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True]))

In [325]:
print(word_char_tokenizer.char_to_idx.keys())

dict_keys(['-', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я', '<eos>', '<pad>', '<unk>', '<sos>'])


In [326]:
len(word_char_tokenizer.tokenize('информационно-телекоммуникационной')[0])

36

In [327]:
len(word_char_tokenizer.tokenize('информационно')[0])

36

In [328]:
word_char_tokenizer.decode(word_char_tokenizer.tokenize('информационно-телекоммуникационной')[0])

'<sos>информационно-телекоммуникационной<eos>'

In [329]:
word_char_tokenizer.max_word_len - 1

35

In [430]:
def prepare_batch_v1(x, y, device):
    (xyt, kb_tokens, dec_in_char_seq, traj_pad_mask, word_pad_mask), dec_out_char_seq = x, y
    
    if xyt is not None:
        xyt = xyt.transpose_(0, 1).to(device)  # (curves_seq_len, batch_size, n_coord_feats)
    if kb_tokens is not None:
        kb_tokens = kb_tokens.transpose_(0, 1).to(device) # (curves_seq_len, batch_size)
    if dec_in_char_seq is not None:
        dec_in_char_seq = dec_in_char_seq.transpose_(0, 1).to(device)  # (chars_seq_len - 1, batch_size)
    if dec_out_char_seq is not None:
        dec_out_char_seq = dec_out_char_seq.transpose_(0, 1).to(device)  # (chars_seq_len - 1, batch_size)

    if traj_pad_mask is not None:
        traj_pad_mask = traj_pad_mask.to(device)  # (batch_size, curves_seq_len)
    if word_pad_mask is not None:
        word_pad_mask = word_pad_mask.to(device)  # (batch_size, chars_seq_len - 1)

    return (xyt, kb_tokens, dec_in_char_seq, traj_pad_mask, word_pad_mask), dec_out_char_seq

prepare_batch = prepare_batch_v1

In [445]:
def turncate_traj_batch(xyt, kb_tokens, traj_pad_mask):
    max_curve_len = int(torch.max(torch.sum(~traj_pad_mask, dim = 1)))
    xyt = xyt[:, :max_curve_len]
    kb_tokens = kb_tokens[:, :max_curve_len]
    traj_pad_mask = traj_pad_mask[:, :max_curve_len]
    return xyt, kb_tokens, traj_pad_mask



def prepare_batch_with_pad_truncation(x, y, device):
    (xyt, kb_tokens, dec_in_char_seq, traj_pad_mask, word_pad_mask), dec_out_char_seq = x, y

    xyt, kb_tokens, traj_pad_mask = turncate_traj_batch(xyt, kb_tokens, traj_pad_mask)

    # print(max_curve_len)

    xyt = xyt.transpose_(0, 1).to(device)  # (curves_seq_len, batch_size, n_coord_feats)
    kb_tokens = kb_tokens.transpose_(0, 1).to(device) # (curves_seq_len, batch_size)
    dec_in_char_seq = dec_in_char_seq.transpose_(0, 1).to(device)  # (chars_seq_len - 1, batch_size)
    dec_out_char_seq = dec_out_char_seq.transpose_(0, 1).to(device)  # (chars_seq_len - 1, batch_size)

    traj_pad_mask = traj_pad_mask.to(device)  # (batch_size, max_curve_len)
    # traj_pad_mask = torch.zeros_like(kb_tokens, dtype = torch.bool).transpose_(0, 1).to(device)
    word_pad_mask = word_pad_mask.to(device)  # (batch_size, chars_seq_len - 1)

    return (xyt, kb_tokens, dec_in_char_seq, traj_pad_mask, word_pad_mask), dec_out_char_seq

# prepare_batch = prepare_batch_with_pad_truncation

In [378]:
max_out_seq_len = word_char_tokenizer.max_word_len - 1

transformer = SwipeCurveTransformer(
    n_coord_feats=6,
    char_emb_size=128,
    char_vocab_size=len(word_char_tokenizer.char_to_idx),
    key_emb_size=54,
    num_encoder_layers=4,
    num_decoder_layers=3,
    dim_feedforward=128,
    num_heads_encoder_1=4,
    num_heads_encoder_2=4,
    num_heads_decoder=4,
    dropout=0.1,
    char_embedding_dropout=0.1,
    key_embedding_dropout=0.1,
    max_out_seq_len=max_out_seq_len,
    max_curves_seq_len=299,
    device = device)

In [379]:
import torch.nn.functional as F

def cross_entropy_with_reshape(pred, target):
    """
    pred - BatchSize x TargetLen x VocabSize
    target - BatchSize x TargetLen
    """
    pred_flat = pred.view(-1, pred.shape[-1])  # BatchSize*TargetLen x VocabSize
    target_flat = target.reshape(-1)  # BatchSize*TargetLen
    return F.cross_entropy(pred_flat, target_flat, ignore_index=0)

In [380]:
from torch.utils.data import DataLoader

loader = DataLoader(dataset, batch_size=10, shuffle=True)


for x, y in loader:
    x, y = prepare_batch(x, y, device)

    char_seq_pred = transformer(*x)

    cross_entropy_with_reshape(char_seq_pred, y)

    
    break

c:\Users\proshian\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


In [381]:
char_seq_pred.transpose(0,1)[0].shape

if True in torch.isnan(char_seq_pred):
    print(char_seq_pred)

In [382]:
print(char_seq_pred.shape)  # max_word_len - 1, batch_size, char_vocab_size - 2 (omitting <sos> and <pad>)

torch.Size([35, 10, 35])


In [383]:
def lr_scheduler(optimizer):
    return torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                      patience=20,
                                                      factor=0.5,
                                                      verbose=True)

In [384]:
def init_random_seed(value=42):
    # random.seed(value)
    np.random.seed(value)
    torch.manual_seed(value)
    torch.cuda.manual_seed(value)
    # torch.backends.cudnn.deterministic = True

In [425]:
class GreedyGenerator:
    def __init__(self, model, tokenizer, device):
        self.model = model
        self.tokenizer = tokenizer
        self.device = torch.device(device)
        self.model.to(self.device)
        self.eos_token_id = tokenizer.char_to_idx['<eos>'] 

    def __call__(self, xyt, kb_tokens, traj_pad_mask, max_steps_n=35):
        tokens = [self.tokenizer.char_to_idx['<sos>']]

        # We don't have to put everything to device because it's done in prepare_batch.

        with torch.no_grad():
            for _ in range(max_steps_n):
                dec_in_char_seq = torch.tensor(tokens).to(self.device)
                word_pad_mask = torch.zeros_like(dec_in_char_seq, dtype=torch.bool, device=self.device)
                # dummy_y is any tensor with n_dims = 2 (chars_seq_len - 1, batch_size).
                dummy_y = torch.tensor([[1]])
                x = [xyt, kb_tokens, dec_in_char_seq, traj_pad_mask, word_pad_mask]
                x = [el.unsqueeze(0) for el in x]
                model_input, dummy_y = prepare_batch(x, dummy_y, self.device)
                best_next_token = self.model(*model_input).transpose_(0, 1)
                best_next_token = best_next_token[0, -1].argmax()  # batch_i = 0, decoder_out_onehot_vector_seq_i = -1 
                if best_next_token == self.eos_token_id:
                    break

                tokens.append(int(best_next_token))
        
        return self.tokenizer.decode(tokens)

In [393]:
# class GreedyGeneratorBatched:
#     def __init__(self, model, tokenizer, device):
#         self.model = model
#         self.tokenizer = tokenizer
#         self.device = torch.device(device)
#         self.model.to(self.device)
#         self.eos_token_id = tokenizer.char_to_idx['<eos>'] 

#     def __call__(self, xyt, kb_tokens, traj_pad_mask, max_steps_n=35):
#         curve_seq_len, batchsize, emb_size = xyt.shape
#         tokens = [[self.tokenizer.char_to_idx['<sos>']] for _ in range(batchsize)]

#         xyt = xyt.to(self.device)
#         kb_tokens = kb_tokens.to(self.device)
#         traj_pad_mask = traj_pad_mask.to(self.device)

#         with torch.no_grad():

#             for _ in range(max_steps_n):
#                 dec_in_char_seq = torch.tensor(tokens).to(self.device)
#                 word_pad_mask = torch.zeros_like(dec_in_char_seq, dtype=torch.bool)
#                 # (seqlen, batchsize) but it's ok to have wrong sizes, what important is n_dims
#                 dummy_y = torch.zeros(1, 1, device = self.device)  
#                 x = (xyt, kb_tokens, dec_in_char_seq, traj_pad_mask, word_pad_mask)
#                 model_input, dummy_y = prepare_batch(x, dummy_y, self.device)
#                 best_next_tokens = self.model(*model_input).transpose_(0, 1)  # seq_len, batch_size
                
#                 best_next_token = best_next_token[0, -1].argmax()
#                 if best_next_token == self.eos_token_id:
#                     break

#                 tokens.append(int(best_next_token))
        
#         return self.tokenizer.decode(tokens)

In [394]:
model = SwipeCurveTransformer(
    n_coord_feats=6,
    char_emb_size=128,
    char_vocab_size=len(word_char_tokenizer.char_to_idx),
    key_emb_size=54,
    num_encoder_layers=4,
    num_decoder_layers=3,
    dim_feedforward=128,
    num_heads_encoder_1=4,
    num_heads_encoder_2=4,
    num_heads_decoder=4,
    dropout=0.1,
    char_embedding_dropout=0.1,
    key_embedding_dropout=0.1,
    max_out_seq_len=max_out_seq_len,
    max_curves_seq_len=299,
    device = device)

In [395]:
model.load_state_dict(
    torch.load(os.path.join(MODELS_DIR,
                            'best_model__2023_11_04__18_31_37__0.02530_default_switch_2.pt'),
               map_location = device))

model = model.eval()

In [396]:
greedy_generator = GreedyGenerator(model, word_char_tokenizer, device)


print("{:<20} {:<20}".format("target", "prediction"))
print("-"*31)

n_examples = 40
for i in range(n_examples):

    (xyt, kb_tokens, dec_in_char_seq, traj_pad_mask, word_pad_mask), target = dataset[i]

    pred = greedy_generator(xyt, kb_tokens, traj_pad_mask)

    # strip работвет только потому что в настоящих словах нет этих символов
    pred = pred.strip("<eos><pad>") 
    target = word_char_tokenizer.decode(target).strip("<eos><pad>")
    print("{:<20} {:<20}".format(target, pred))

target               prediction          
-------------------------------
на                   на                  
все                  все                 
добрый               добрый              
девочка              девочка             
сказала              сказала             
скинь                скинь               
геев                 гееев               
тобой                тобой               
была                 быстра              
да                   да                  
муж                  маж                 
щас                  щас                 
она                  она                 
проблема             проблема            
билайн               билайн              
уже                  уже                 
раньше               раньше              
рам                  нам                 
щас                  щас                 
купил                купил               
ты                   ты                  
зовут                зовут               
ко

In [397]:
grid_name_to_grid_path = os.path.join(DATA_ROOT, "gridname_to_grid.json")
with open(grid_name_to_grid_path, "r", encoding="utf-8") as f:
    grid_name_to_grid = json.load(f)

test_path = os.path.join(DATA_ROOT, f"test.jsonl")

kb_tokenizer = KeyboardTokenizerv1()
word_tokenizer = CharLevelTokenizerv1(os.path.join(DATA_ROOT, "voc.txt"))


test_dataset = NeuroSwipeDatasetv1(
    data_path = test_path,
    gridname_to_grid = grid_name_to_grid,
    kb_tokenizer = kb_tokenizer,
    max_traj_len = 299,
    word_tokenizer = word_tokenizer,
    include_time = False,
    include_velocities = True,
    include_accelerations = True,
    has_target=False,
    has_one_grid_only=False,
    include_grid_name=True,
    total = 10_000
)

100%|██████████| 10000/10000 [00:01<00:00, 5438.07it/s]


In [353]:
grid_name_to_grid_path = os.path.join(DATA_ROOT, "gridname_to_grid.json")
with open(grid_name_to_grid_path, "r", encoding="utf-8") as f:
    grid_name_to_grid = json.load(f)

val_path = os.path.join(DATA_ROOT, f"valid__in_train_format.jsonl")

kb_tokenizer = KeyboardTokenizerv1()
word_tokenizer = CharLevelTokenizerv1(os.path.join(DATA_ROOT, "voc.txt"))


val_dataset = NeuroSwipeDatasetv1(
    data_path = val_path,
    gridname_to_grid = grid_name_to_grid,
    kb_tokenizer = kb_tokenizer,
    max_traj_len = 299,
    word_tokenizer = word_tokenizer,
    include_time = False,
    include_velocities = True,
    include_accelerations = True,
    has_target=False,
    has_one_grid_only=False,
    include_grid_name=True,
    total = 10_000
)

100%|██████████| 10000/10000 [00:02<00:00, 3654.97it/s]


In [399]:
def model_getter(weights_path):
    model = SwipeCurveTransformer(
    n_coord_feats=6,
    char_emb_size=128,
    char_vocab_size=len(word_char_tokenizer.char_to_idx),
    key_emb_size=54,
    num_encoder_layers=4,
    num_decoder_layers=3,
    dim_feedforward=128,
    num_heads_encoder_1=4,
    num_heads_encoder_2=4,
    num_heads_decoder=4,
    dropout=0.1,
    char_embedding_dropout=0.1,
    key_embedding_dropout=0.1,
    max_out_seq_len=max_out_seq_len,
    max_curves_seq_len=299,
    device = device)

    model.load_state_dict(
        torch.load(weights_path,
                map_location = device))
    
    model = model.eval()

    return model

In [400]:
grid_name_to_model = {
    "default": model_getter(os.path.join(MODELS_DIR,
                                         "best_model__2023_11_04__18_31_37__0.02530_default_switch_2.pt")),
    "extra": model_getter(os.path.join(MODELS_DIR,
                                       "best_model__2023_11_05__07_55_13__0.02516_extra_switch_2__with_pad_cutting.pt"))
}

In [401]:
grid_name_to_greedy_generator_v1 = {
    grid_name: GreedyGenerator(grid_name_to_model[grid_name], word_char_tokenizer, device)
    for grid_name in ("default", "extra")
}

In [402]:
# test_default_predictions = []

# for i in tqdm(range(len(test_dataset)), position=0, leave=True):

#     (xyt, kb_tokens, dec_in_char_seq, traj_pad_mask, word_pad_mask), target, grid_name = test_dataset[i]

#     pred = grid_name_to_greedy_generator[grid_name](xyt, kb_tokens, traj_pad_mask)

#     # strip работвет только потому что в настоящих словах нет этих символов
#     pred = pred.removeprefix("<sos>") 
#     test_default_predictions.append(pred)

#     if i > 50:
#         break


In [403]:
def create_submission_greedy(dataset,
                             grid_name_to_greedy_generator,
                             baseline_preds_path, 
                             out_path,
                             vocab_set):
    """
    Creates submission file generating words greedily.

    If prediction is not in the vocabulary 
    """

    if os.path.exists(out_path):
        raise ValueError(f"File {out_path} already exists")
    
    with open(out_path, "w", encoding="utf-8") as f:
        with open(baseline_preds_path, "r", encoding="utf-8") as baseline_f:
            for i, (data, baseline_str) in tqdm(enumerate(zip(dataset, baseline_f)), total=len(dataset)):
                (xyt, kb_tokens, _, traj_pad_mask, _), _, grid_name = data
                pred = grid_name_to_greedy_generator[grid_name](xyt, kb_tokens, traj_pad_mask)
                pred = pred.removeprefix("<sos>") 
                if pred in vocab_set:
                    pred_list = [pred, *baseline_str.split(",")[:-1]]
                    pred_str = ",".join(pred_list)
                    f.write(pred_str + "\n")
                else:
                    f.write(baseline_str)

In [404]:
def create_pred_list_beam(dataset,
                          grid_name_to_beam_generator,
                          beamsize,
                          verbose):
    """
    Creates submission file generating words greedily.

    If prediction is not in the vocabulary 
    """
    all_pred_list = [None]*len(dataset)

    for i, data in tqdm(enumerate(dataset), total=len(dataset)):
        try:
            (xyt, kb_tokens, _, traj_pad_mask, _), _, grid_name = data
            pred_list = grid_name_to_beam_generator[grid_name](
                xyt, kb_tokens, traj_pad_mask, max_steps_n = 35, beamsize=beamsize, verbose=verbose)
            pred_list = [pred for score, pred in pred_list]
            all_pred_list[i] = pred_list
            
        except KeyboardInterrupt:
            print('Досрочно остановлено пользователем')
            break
    return all_pred_list

In [405]:
def create_submission(preds_list, out_path):
    if os.path.exists(out_path):
        raise ValueError(f"File {out_path} already exists")
    
    with open(out_path, "w", encoding="utf-8") as f:
        for preds in preds_list:
            pred_str = ",".join(preds)
            f.write(preds + "\n")

In [406]:
def get_vocab_set(vocab_path: str):
    with open(vocab_path, 'r', encoding = "utf-8") as f:
        return set(f.read().splitlines())

In [407]:
vocab_set = get_vocab_set(os.path.join(DATA_ROOT, "voc.txt"))

In [416]:
prepare_batch = prepare_batch_v1

In [451]:
import heapq



class BeamGenerator:
    def __init__(self, model, tokenizer, voc_set, device):
        self.model = model
        self.tokenizer = tokenizer
        self.voc_set = voc_set
        self.device = torch.device(device)
        self.model.to(self.device)
        self.eos_token_id = tokenizer.char_to_idx['<eos>']

    def __call__(self,
                 xyt, kb_tokens, traj_pad_mask,
                 max_steps_n=40,  # max tokens in a seq
                 return_hypotheses_n=4,  # n best hypothesis to return
                 beamsize=6,  # n best solutions we store in intermidiate comuptations
                 normalization_factor=0.5,
                 verbose=True
                 ):
        with torch.no_grad():
            
            tokens = [self.tokenizer.char_to_idx['<sos>']]
            initial_length = len(tokens)

            # Partial hypothesis is a heap (stored as a list) of tuples.
            # Each tuple consists of a partial (unfinishedaka intermidiate)
            # hypothesis and it's weight.
            # Weight is a measure of likelihood of the hypothesis.
            # [(w1, hypothesis1), (w2, hypothesis2), ...] 
            partial_hypotheses = [(0, tokens)]
            final_hypotheses = []


            xyt, kb_tokens, traj_pad_mask = (el.unsqueeze(0) for el in (xyt, kb_tokens, traj_pad_mask))
            xyt, kb_tokens, traj_pad_mask = turncate_traj_batch(xyt, kb_tokens, traj_pad_mask)
            xyt, kb_tokens, traj_pad_mask = (el.to(device) for el in (xyt, kb_tokens, traj_pad_mask))
            xyt, kb_tokens = (el.transpose(0, 1) for el in (xyt, kb_tokens))


            encoded = self.model.encode(xyt, kb_tokens, traj_pad_mask)

            while len(partial_hypotheses) > 0:
                cur_partial_score, cur_partial_hypothesis = heapq.heappop(partial_hypotheses)


                dec_in_char_seq = torch.tensor(cur_partial_hypothesis).unsqueeze(0).to(device)
                word_pad_mask = torch.zeros_like(dec_in_char_seq, dtype=torch.bool, device=self.device)
                dec_in_char_seq.transpose_(0,1)

                
                next_tokens_logits = self.model.decode(encoded, dec_in_char_seq, traj_pad_mask, word_pad_mask).transpose_(0, 1)[0, -1]
                next_tokens_logproba = F.log_softmax(next_tokens_logits)
                topk_continuations = next_tokens_logproba.topk(beamsize)

                for token_score, token_idx in zip(topk_continuations.values, topk_continuations.indices):
                    # Convert tesors to loat and int to avoid memory leakage.
                    token_score = float(token_score)
                    token_idx = int(token_idx)

                    # score - нормализованная разность log_softmax всех токенов.
                    # Разность, а не сумма, потому что heapq - мин-куча. 
                    old_denorm_score = cur_partial_score * len(cur_partial_hypothesis)**normalization_factor
                    new_score = (old_denorm_score - token_score) / (len(cur_partial_hypothesis) + 1)**normalization_factor

                    new_hypothesis = cur_partial_hypothesis + [token_idx]
                    new_item = (new_score, new_hypothesis)

                    if token_idx == self.eos_token_id or len(new_hypothesis) - initial_length >= max_steps_n:
                        final_hypotheses.append(new_item)
                    else:
                        heapq.heappush(partial_hypotheses, new_item)

                if len(partial_hypotheses) > beamsize:
                    partial_hypotheses = heapq.nsmallest(beamsize, partial_hypotheses)
                    heapq.heapify(partial_hypotheses)

            final_scores, final_token_lists = zip(*final_hypotheses)
            final_texts = [self.tokenizer.decode(final_token_list) for final_token_list in final_token_lists]
            result = list(zip(final_scores, final_texts))
            result.sort()

            if verbose:
                print(result)

            clean_result = []
            for score, word in result:
                word = word.removeprefix("<sos>").removesuffix("<eos>")
                if word not in self.voc_set:
                    continue
                clean_result.append((score, word))
                if len(clean_result) == return_hypotheses_n:
                    break
            # result = result[:return_hypotheses_n]

            if verbose:
                print(clean_result)

            return clean_result

In [452]:
grid_name_to_beam_generator = {
    grid_name: BeamGenerator(grid_name_to_model[grid_name], word_char_tokenizer, vocab_set, device)
    for grid_name in ("default", "extra")
}

test_preds = create_pred_list_beam(
    test_dataset,
    grid_name_to_beam_generator,
    beamsize = 4,
    verbose = False
)

  0%|          | 0/10000 [00:00<?, ?it/s]C:\Users\proshian\AppData\Local\Temp\ipykernel_400\478798116.py:52: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  next_tokens_logproba = F.log_softmax(next_tokens_logits)
  0%|          | 7/10000 [00:10<4:06:42,  1.48s/it]

Досрочно остановлено пользователем


In [450]:
test_preds[:18]

[['на', 'неа', 'не', 'нас'],
 ['что', 'сто', 'чтоб', 'сито'],
 ['опоздания', 'опоздание', 'опоздании', 'опоздает'],
 ['сколько', 'скольки', 'скольких', 'сколь'],
 ['дремать', 'донимать'],
 ['не', 'нее', 'ну', 'на'],
 ['как', 'кака', 'какие', 'како'],
 ['садовод', 'садов'],
 ['заметил', 'заметили', 'заметила', 'заметит'],
 ['ваши', 'ваги', 'аги', 'вани'],
 ['ок', 'он', 'оке', 'ока'],
 [],
 ['ай', 'ау', 'айк', 'айкай'],
 ['ищем', 'ищет', 'идем', 'идет'],
 ['они', 'лет', 'он', 'оно'],
 None,
 None,
 None]

In [415]:
test_preds[:13]

[['на', 'неа', 'не', 'нас'],
 ['что', 'сто', 'чтоб', 'сито'],
 ['опоздания', 'опоздание', 'опоздании', 'опоздает'],
 ['сколько', 'скольки', 'скольких', 'сколь'],
 ['дремать', 'донимать'],
 ['не', 'нее', 'ну', 'на'],
 ['как', 'кака', 'какие', 'како'],
 ['садовод', 'садов'],
 ['заметил', 'заметили', 'заметила', 'заметит'],
 ['ваши', 'ваги', 'аги', 'вани'],
 ['ок', 'он', 'оке', 'ока'],
 [],
 ['ай', 'ау', 'айк', 'айкай']]

In [109]:
grid_name_to_beam_generator = {
    grid_name: BeamGenerator(grid_name_to_model[grid_name], word_char_tokenizer, vocab_set, device)
    for grid_name in ("default", "extra")
}

val_preds = create_pred_list_beam(
    val_dataset,
    grid_name_to_beam_generator,
    beamsize = 4,
    verbose = False
)

  0%|          | 0/10000 [00:00<?, ?it/s]/tmp/ipykernel_32/1800239837.py:45: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  next_tokens_logproba = F.log_softmax(next_tokens_logits)
  1%|          | 87/10000 [01:34<3:00:17,  1.09s/it]

Досрочно остановлено пользователем


In [110]:
val_preds[:34]

[['на', 'нас', 'неа', 'нан'],
 ['все', 'всем', 'всех', 'всей'],
 ['этом', 'этому', 'этот', 'этим'],
 ['добрый', 'добрые', 'добрым'],
 ['девочка', 'девочки', 'девочку', 'девочке'],
 ['сказала', 'сказал', 'сказали', 'сказало'],
 ['скинь', 'скин'],
 ['геев'],
 ['тобой'],
 ['быстра', 'баса', 'быстро', 'быстр'],
 ['есть', 'если', 'ест', 'ес'],
 ['да', 'дора', 'дав', 'дал'],
 ['муж', 'мало', 'малом'],
 ['щас', 'щам', 'ща'],
 ['она', 'они', 'оная', 'он'],
 ['проблема', 'проблемы', 'проблему', 'проблем'],
 ['билайн'],
 ['уже', 'ужен', 'ужа', 'ужи'],
 ['раньше'],
 ['нам', 'рам', 'нас', 'рас'],
 ['щас', 'ща', 'щам'],
 ['купил', 'купила', 'купили', 'купило'],
 ['ты', 'бы', 'тв', 'та'],
 ['зовут', 'хрупки', 'хочу', 'хрупать'],
 ['короче', 'корочек', 'корочке', 'короч'],
 ['лучше', 'лучшее', 'лучшей', 'лучшие'],
 ['приедем', 'придем', 'приедет', 'приеду'],
 ['размыто', 'размыть', 'размывать', 'размыт'],
 ['давай', 'давайте'],
 ['ты', 'тв', 'та', 'тыс'],
 ['отдать', 'отжать', 'отдаю', 'отдают'],
 ['

In [111]:
with open(os.path.join(DATA_ROOT, "valid.ref"), 'r', encoding = "utf-8") as f:
    val_reference = f.read().splitlines()
val_reference[:34]

['на',
 'все',
 'этом',
 'добрый',
 'девочка',
 'сказала',
 'скинь',
 'геев',
 'тобой',
 'была',
 'есть',
 'да',
 'муж',
 'щас',
 'она',
 'проблема',
 'билайн',
 'уже',
 'раньше',
 'рам',
 'щас',
 'купил',
 'ты',
 'зовут',
 'короче',
 'лучше',
 'приедем',
 'размыто',
 'давай',
 'ты',
 'отдать',
 'привет',
 'не',
 'да']

при beamsize = 4 на Kaggle:
примерно 4 часа без P100 и 3 часа с P100
Если использовать prepare_batch_with_pad_clipping часа

In [51]:
# from concurrent.futures import ProcessPoolExecutor
# from functools import partial

# def predict_example(data, grid_name_to_greedy_generator):
#     i, ((xyt, kb_tokens, _, traj_pad_mask, _), _, grid_name) = data
#     generator = grid_name_to_greedy_generator[grid_name]
#     pred = generator(xyt, kb_tokens, traj_pad_mask)
#     pred = pred.removeprefix("<sos>")
#     return i, pred

# def get_model_predictions(dataset,
#                           grid_name_to_greedy_generator,
#                           num_workers=2):
#     """
#     Creates submission file generating words greedily.

#     If prediction is not in the vocabulary 
#     """
#     predictions = [None] * len(dataset)
    
#     g2gg = grid_name_to_greedy_generator
    
#     with ProcessPoolExecutor(num_workers) as executor:
#         process_function = partial(predict_example, grid_name_to_greedy_generator=g2gg)
#         for idx, result in tqdm.tqdm(executor.map(process_function, enumerate(dataset)), total=len(dataset)):
#                 predictions[idx] = result
    
#     return predictions

In [52]:
# class ParallelProcessor:
#     def __init__(self, num_workers):
#         self.num_workers = num_workers

#     def process(self, data, process_function, *additional_args):
#         results = [None] * len(data)
#         with ProcessPoolExecutor(max_workers=self.num_workers) as executor: 
#             process_function = partial(process_function, *additional_args)
#             for idx, result in tqdm.tqdm(executor.map(process_function, enumerate(data)), total=len(data)):
#                 results[idx] = result

#         return results

In [53]:
# from multiprocessing import Pool

# def get_model_predictions(dataset, grid_name_to_greedy_generator, num_workers=4):
#     predictions = [None] * len(dataset)
#     g2gg = grid_name_to_greedy_generator

#     with Pool(num_workers) as pool:
#         process_function = partial(predict_example, grid_name_to_greedy_generator=g2gg)
#         for idx, result in tqdm(pool.imap(process_function, enumerate(dataset)), total=len(dataset)):
#             predictions[idx] = result

#     return predictions

In [152]:
# model_preds = get_model_predictions(
#     test_dataset,
#     grid_name_to_greedy_generator_v1,
#     num_workers=2
# )

NameError: name 'get_model_predictions' is not defined

In [74]:
from tqdm import tqdm

The sell below is commented out until i find out if computation results are the same as when we use regular prepare_batch

In [73]:
grid_name_to_greedy_generator = {
    grid_name: GreedyGenerator(grid_name_to_model[grid_name], word_char_tokenizer, device)
    for grid_name in ("default", "extra")
}

In [76]:
create_submission_greedy(
    test_dataset,
    grid_name_to_greedy_generator,
    baseline_preds_path = "../data/submissions/sample_submission.csv",
    out_path = "greedy_submionvdrp.csv",
    vocab_set = vocab_set
)

  0%|          | 4/10000 [00:03<2:40:03,  1.04it/s]


KeyboardInterrupt: 

In [ ]:
def remove_duplicates(preds):
    new_preds = []
    met_preds = {}
    for pred in preds:
        if pred in met_preds:
            continue
        met_preds.add(pred)
        new_preds.append(pred)
    return new_preds

def get_metric(preds_list, ref):
    # Works properly if has duplicates or n_line_preds < 4

    MMR = 0
    
    for preds, target in zip(preds_list, ref):
        preds = remove_duplicates(preds)

        weights = [1, 0.1, 0.09, 0.08]

        line_MRR = sum(weights[i]*pred for i, pred in enumerate(preds))

        MMR += line_MRR
    
    MMR /= len(preds_list)

    return MMR